# RAG Fusion

In [ ]:
!pip -q install langchain huggingface_hub openai tiktoken pypdf
!pip -q install google-generativeai chromadb unstructured
!pip -q install sentence_transformers
!pip -q install -U FlagEmbedding


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Download the Data & Utils

In [ ]:
import os
import requests
import zipfile
from io import BytesIO
import textwrap

def download_and_extract_zip(url, target_folder):
    # Ensure the target folder exists
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Download the file from the URL
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {url}")

    # Unzip the file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(target_folder)

    print(f"Files extracted to {target_folder}")


def zip_folder(folder_path, zip_file_path):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the directory
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a path relative to the folder to avoid storing absolute paths
                relative_path = os.path.relpath(os.path.join(root, file), os.path.dirname(folder_path))
                # Add file to the zip file
                zipf.write(os.path.join(root, file), arcname=relative_path)

    print(f"{zip_file_path} created successfully.")



def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

# URL of the zip file
url = "https://www.dropbox.com/scl/fi/av3nw07o5mo29cjokyp41/singapore_text_files_languages.zip?rlkey=xqdy5f1modtbnrzzga9024jyw&dl=1" # Ensure dl=1 for direct download

# Folder to save extracted files
folder = "singapore_text"

# Call the function
download_and_extract_zip(url, folder)


Files extracted to singapore_text


In [ ]:
## download the chroma DB
url = 'https://www.dropbox.com/scl/fi/3kep8mo77h642kvpum2p7/singapore_chroma_db.zip?rlkey=4ry4rtmeqdcixjzxobtmaajzo&dl=1'
download_and_extract_zip(url, '.')

Files extracted to .


In [ ]:
import os

from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.338
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Google

## Imports

In [ ]:
from langchain.llms import GooglePalm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
import langchain



## Load in Docs

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [ ]:
%%time
loader = DirectoryLoader('/content/singapore_text/Textfiles3/English/', glob="*.txt", show_progress=True)
docs = loader.load()

  0%|          | 0/646 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 646/646 [00:45<00:00, 14.14it/s]

CPU times: user 41.2 s, sys: 1.18 s, total: 42.4 s
Wall time: 45.7 s


In [ ]:
len(docs)
# docs = docs[:10]
len(docs)

646

In [ ]:
docs[0]

Document(page_content='Link: https://www.visitsingapore.com/see\n\ndo\n\nsingapore/nightlife/\n\nTitle: Nightlife', metadata={'source': '/content/singapore_text/Textfiles3/English/Nightlife.txt'})

In [ ]:
raw_text = ''
for i, doc in enumerate(docs):
    text = doc.page_content
    if text:
        raw_text += text

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex = False,
)

In [ ]:
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

6201

In [ ]:
texts[1]

"chefs/\n\nTitle: Chefs in Singapore to check out\n\nSingapore is known as a foodie's city, where people are enthusiastic about all things gastronomic, be it street eats or fine dining. As a global metropolis, our city’s passionate chefs draw on culinary influences and inspiration from all across the region and the globe, making every meal in Singapore a unique feast."

## BGE Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs,

)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

## Vector DB

In [ ]:
%%time
### Make the chroma and persiste to disk
# db = Chroma.from_texts(texts,
#                        embedding_function,
#                        persist_directory="./chroma_db")



### load from disk
db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)





CPU times: user 413 ms, sys: 28 ms, total: 441 ms
Wall time: 515 ms


In [ ]:
### Save to zip
# zip_folder('/content/chroma_db', 'chroma_db.zip')

chroma_db.zip created successfully.


In [ ]:
query = "Tell me about Universal Studios Singapore?"

db.similarity_search(query, k=5)

[Document(page_content='Universal Studios Singapore'),
 Document(page_content='Universal Studios Singapore is a popular theme park that offers thrilling rides and entertainment for all ages. You can buy Universal Studios Singapore tickets and enjoy an unbeatable discount of 40% from MySingaporePass. The pass provides a hassle-free booking process, allowing you to skip the long queues and gain easy access to the park. Enjoy exhilarating roller coasters, live shows, and immersive themed zones inspired by your favorite movies. The discounted entry with My Singapore Pass'),
 Document(page_content='leisure/fun\n\nthings\n\nto\n\ndo/universal\n\nstudios\n\nsingapore/\n\nTitle: Universal Studios Singapore\n\nThe shimmering wonders of the silver screen comes to vivid life at Universal Studios Singapore, the first-ever Hollywood movie theme park in Southeast Asia.'),
 Document(page_content='The magic of Hollywood comes to life at Universal Studios Singapore (USS), a theme park filled with adren

## Setup a Retriever

In [ ]:
retriever = db.as_retriever(k=5) # can add mmr fetch_k=20, search_type="mmr"

retriever.get_relevant_documents(query)[1]

Document(page_content='Universal Studios Singapore is a popular theme park that offers thrilling rides and entertainment for all ages. You can buy Universal Studios Singapore tickets and enjoy an unbeatable discount of 40% from MySingaporePass. The pass provides a hassle-free booking process, allowing you to skip the long queues and gain easy access to the park. Enjoy exhilarating roller coasters, live shows, and immersive themed zones inspired by your favorite movies. The discounted entry with My Singapore Pass')

## Chat chain

In [ ]:
from operator import itemgetter

from langchain.chat_models import ChatGooglePalm

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

In [ ]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatGooglePalm()

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
text_reply = chain.invoke("Tell me about Universal Studio Singapore")

print(wrap_text(text_reply))

Universal Studios Singapore (USS) is a theme park located at Resorts World Sentosa on
Sentosa Island, Singapore. It is the first Universal Studios theme park in Southeast Asia.
The park opened on 18 September 2010, and is the second Universal Studios theme park to
open in Asia after Universal Studios Japan.

USS is divided into seven themed zones: Hollywood, New York, Sci-Fi City, Ancient Egypt,
The Lost World, Far Far Away, and Madagascar. Each zone is home to a variety of rides,
shows, and attractions based on popular movies and television shows.

Some of the most popular rides at USS include:

* The Mummy Ride: A roller coaster that takes guests on a journey through the tomb of
Imhotep.
* Transformers The Ride: 3D Battle: A 3D motion simulator ride that puts guests in the
middle of a battle between the Autobots and the Decepticons.
* Jurassic Park Rapids Adventure: A water ride that takes guests on a journey through the
Jurassic Park lagoon.
* Shrek 4-D Adventure: A 4D movie experie

## With RagFusion

In [ ]:
from langchain.chat_models import ChatGooglePalm
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

In [ ]:
prompt = ChatPromptTemplate(
              input_variables=['original_query'],
              messages=[
                  SystemMessagePromptTemplate(
                      prompt=PromptTemplate(input_variables=[],
                                            template='You are a helpful assistant that generates multiple search queries based on a single input query.')),
                  HumanMessagePromptTemplate(
                      prompt=PromptTemplate(input_variables=['original_query'],
                                            template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))
                  ]
              )

In [ ]:
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant that generates multiple search queries based on a single input query."),
#     ("user", "Generate multiple search queries related to: {question}/n OUTPUT (4 queries):"),
# ])

prompt

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generates multiple search queries based on a single input query.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])

In [ ]:
generate_queries = (
    prompt | ChatGooglePalm(temperature=0) | StrOutputParser() | (lambda x: x.split("\n"))
)

In [ ]:
original_query = "universal studios Singapore"

In [ ]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [ ]:
ragfusion_chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [ ]:
langchain.debug = True

In [ ]:
ragfusion_chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [ ]:
ragfusion_chain.invoke({"question": original_query})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "universal studios Singapore"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "universal studios Singapore"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "SystemMessage"
        ],
        "kwargs": {
          "content": "You are a helpful assistant that generates multiple search queries based on a single input query.",
          "additional_kwargs": {}
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          

[(Document(page_content='Universal Studios Singapore'), 0.06639344262295083),
 (Document(page_content='Must-visit attractions such as the Universal Studios Singapore are probably already on your itinerary, but be sure not to miss out on kids’ priced tickets (one-day passes are S$56 for children aged four to 12) and kids’ events while you’re there. Start your day sunny when you get to Sesame Street—have breakfast with your favourite characters from the show, pose for family photos and play plenty of fun games.Link: https://www.visitsingapore.com/see\n\ndo\n\nsingapore/history/history\n\nmuseums/asian'),
  0.06531817380574652),
 (Document(page_content='PIN IT FOR LATER!\n\nHover over the image below, and press the red “Save” button that pops up:\n\nDo you like pancakes? Would you eat it if you went to Tokyo, or would you just eat Japanese food? Please share your thoughts in the comment area below. If you enjoyed this post and found it useful, please share it on social media! Thanks! 🙂\n\

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
full_rag_fusion_chain.input_schema.schema()

{'title': 'RunnableParallelInput',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [ ]:
full_rag_fusion_chain.invoke({"question": "Tell me about Universal Studio Singapore?"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "me

"Universal Studios Singapore is a theme park located on Sentosa Island in Singapore. It is the first Universal Studios theme park in Southeast Asia. The park opened on 18 September 2010.\n\nUniversal Studios Singapore is divided into seven themed zones:\n\n* Hollywood\n* New York\n* Sci-Fi City\n* Ancient Egypt\n* The Lost World\n* Far Far Away\n* Madagascar\n\nThe park features a variety of rides, shows, and attractions based on popular movies and television shows. Some of the most popular rides include:\n\n* TRANSFORMERS The Ride: The Ultimate 3D Battle\n* Jurassic Park Rapids Adventure\n* Battlestar Galactica: HUMAN vs. CYLON™\n* Revenge of the Mummy: The Ride\n* Shrek 4-D Adventure\n* Sesame Street Spaghetti Space Chase\n\nThe park also features a number of live shows, including:\n\n* Universal's Animal Actors\n* The Universal's Superstar Parade\n* The Universal's Cinematic Spectacular: 100 Years of Movie Memories\n\nUniversal Studios Singapore is a popular tourist destination and 